In [80]:
# Imports for data analysis
import pandas as pd
import requests
import matplotlib.pyplot as pyplot



In [81]:
# read csv
df_crash = pd.read_csv("../Resources/combined_data.csv")
df_crash = df_crash.sort_values(by = "Case Date")
df_crash.head()

/var/folders/zy/55ss7q015wg6ljyvf2xsx9dw0000gn/T/ipykernel_86093/1150434435.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crash = pd.read_csv("../Resources/combined_data.csv")


,St Case Num,Age,Sex,Race,Seat Pos,AirBag Deployed,Is Fatal,Is Drug,Is Drunk,BAC,Case Date,Make Name,Model Name,Model Year,State,County
15,450001,27,1,Not a Fatality (not Applicable),"Second Seat, Right Side",False,False,Not Reported,Not Reported,Test Not Given,2017-01-01,Cadillac,Cadillac CTS/CTC,2003,South Carolina,GEORGETOWN (43)
17,130001,54,2,White,"Front Seat, Left Side",True,True,No (drugs not involved),No (Alcohol Not Involved),Test Not Given,2017-01-01,Chevrolet,Chevrolet Cobalt,2008,Georgia,BARROW (13)
4,310001,999,9,Not a Fatality (not Applicable),"Front Seat, Left Side",True,False,Unknown,Unknown (Police Reported),Test Not Given,2017-01-01,Unknown Make,"Unknown Make Unknown (as to automobile, motore...",9999,Nebraska,SCOTTS BLUFF (157)
5,480010,20,1,White,"Front Seat, Left Side",False,True,Not Reported,Not Reported,Test Not Given,2017-01-01,Chevrolet,Chevrolet S-10/T-10 Pickup,2000,Texas,ROBERTSON (395)
6,480006,25,1,White,"Front Seat, Left Side",False,True,No (drugs not involved),Yes (Alcohol Involved),0.182 % BAC,2017-01-01,Ford,Ford Mustang/Mustang II,2004,Texas,JOHNSON (251)


In [82]:
#check length of csv
len(df_crash)

188221

In [83]:
#checking values
df_crash["Sex"].value_counts()

Sex
1    125605
2     57720
9      3376
8      1520
Name: count, dtype: int64

In [84]:
# remove the unknown and weird numbers
df_crash = df_crash[(df_crash["Sex"] == 1) | (df_crash["Sex"] == 2)]

#convert sex names to actual names, male = 1, female = 2
df_crash.loc[:, "Sex"] = df_crash["Sex"].replace({1: "Male", 2: "Female"})
df_crash.head()

/var/folders/zy/55ss7q015wg6ljyvf2xsx9dw0000gn/T/ipykernel_86093/1245826018.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Male' 'Female' 'Male' ... 'Male' 'Male' 'Female']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_crash.loc[:, "Sex"] = df_crash["Sex"].replace({1: "Male", 2: "Female"})


,St Case Num,Age,Sex,Race,Seat Pos,AirBag Deployed,Is Fatal,Is Drug,Is Drunk,BAC,Case Date,Make Name,Model Name,Model Year,State,County
15,450001,27,Male,Not a Fatality (not Applicable),"Second Seat, Right Side",False,False,Not Reported,Not Reported,Test Not Given,2017-01-01,Cadillac,Cadillac CTS/CTC,2003,South Carolina,GEORGETOWN (43)
17,130001,54,Female,White,"Front Seat, Left Side",True,True,No (drugs not involved),No (Alcohol Not Involved),Test Not Given,2017-01-01,Chevrolet,Chevrolet Cobalt,2008,Georgia,BARROW (13)
5,480010,20,Male,White,"Front Seat, Left Side",False,True,Not Reported,Not Reported,Test Not Given,2017-01-01,Chevrolet,Chevrolet S-10/T-10 Pickup,2000,Texas,ROBERTSON (395)
6,480006,25,Male,White,"Front Seat, Left Side",False,True,No (drugs not involved),Yes (Alcohol Involved),0.182 % BAC,2017-01-01,Ford,Ford Mustang/Mustang II,2004,Texas,JOHNSON (251)
12,450001,36,Male,Not a Fatality (not Applicable),"Front Seat, Left Side",False,False,Not Reported,Not Reported,Not Reported,2017-01-01,Cadillac,Cadillac CTS/CTC,2003,South Carolina,GEORGETOWN (43)


In [85]:
#Check Race Values
print(df_crash["Race"].value_counts())
df_crash["Race"].isna().sum()

Race
Not a Fatality (not Applicable)                                                                   23499
White                                                                                             15702
Black                                                                                              3105
Unknown                                                                                             764
American Indian (includes Alaska Native)                                                            336
All other races                                                                                     278
Other Asian or Pacific Islander                                                                      83
Asian Indian                                                                                         75
Filipino                                                                                             57
Multiple Races (individual races not specified; ex. "mixed"

139137

A lot of the races are NaN. Not useful data.

In [86]:
#checking values 
df_crash["Seat Pos"].value_counts()

#only keeps the ones that have a lot of data
df_seat_pos = df_crash[(df_crash["Seat Pos"] == "Front Seat, Left Side")|
                        (df_crash["Seat Pos"] == "Front Seat, Right Side")|
                        (df_crash["Seat Pos"] == "Second Seat, Left Side")|
                        (df_crash["Seat Pos"] == "Second Seat, Right Side")|
                        (df_crash["Seat Pos"] == "Second Seat, Middle")
                       ][["Seat Pos","Is Fatal"]].copy()
df_seat_pos.head()

,Seat Pos,Is Fatal
15,"Second Seat, Right Side",False
17,"Front Seat, Left Side",True
5,"Front Seat, Left Side",True
6,"Front Seat, Left Side",True
12,"Front Seat, Left Side",False
